# Fetch  1min Crypto Price Data 

In [ ]:
__Author__ = 'Firas Obeid'

In [1]:
# Import libraries and dependencies
import pandas as pd
import ccxt
import csv
import os
from pathlib import Path
from dotenv import load_dotenv
import json
from pytz import timezone
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta
import warnings
from pytz import timezone # set timezone
from io import StringIO
warnings.filterwarnings("ignore")

In [4]:
# !pip install python-dotenv
# !pip install ccxt

In [5]:
load_dotenv()

True

In [6]:
# Read the Broker API key &  Secret Key
binance_key = os.getenv('BINANCEUS_KEY')
binance_secret = os.getenv('BINANCEUS_SECRET')

krayken_key = os.getenv('KRAYKEN_KEY')
krayken_secret = os.getenv('KRAKEN_SECRET')

mexc_key = os.getenv('MEXC_KEY')
mexc_secret = os.getenv('MEXC_SECRET')

kucoin_key = os.getenv('KUCOIN_KEY')
kucoin_secret = os.getenv('KUCOIN_SECRET ')

ex1 = "binanceus"
ex2 = "kraken"
ex3 = "mexc"
ex4 = "kucoin"

In [ ]:
ccxt.

#### BinanceUS

In [7]:
exchange_class1 = getattr(ccxt,ex1)
exchange = exchange_class1({'verbose': True})
binanceus = exchange_class1({
    'apiKey': binance_key,
    'secret': binance_secret,
})

#### Kraken

In [8]:
exchange_class2 = getattr(ccxt,ex2)
kraken = exchange_class2({'verbose': True})
kraken = exchange_class2({
    'apiKey': krayken_key,
    'secret': krayken_secret,
})

##### MEXN

In [9]:
exchange_class3 = getattr(ccxt,ex3)
exchange = exchange_class3({'verbose': True})
mexc = exchange_class3({
    'apiKey': mexc_key,
    'secret': mexc_secret,
})

#### Kucoin

In [10]:
exchange_class4 = getattr(ccxt,ex4)
exchange = exchange_class4({'verbose': True})
kucoin = exchange_class4({
    'apiKey': kucoin_key,
    'secret': kucoin_secret,
})

#### Load Tickers & Fetching Code

In [11]:
with open('crypto_tickers.json',errors="ignore") as f:
    data = list(json.load(f))
    print(f"Loaded {len(data)} tickers")

Loaded 8175 tickers


In [12]:
# #only call once per fetching session
# def get_max_time(ex_name):
#     global max_date
#     # chunk = pd.read_csv("Firas_prices_v2.csv", index_col=0, chunksize=1000000)
#     # df =  pd.concat(chunk)
#     df = pd.read_csv("Firas_prices_v2.1.csv", index_col=0)[["timestamp","exchange"]]
#     try:
#         if len(df[df.exchange == ex_name]) > 0:
#             max_date = df[df.exchange == ex_name].timestamp.max()
#         else:
#             max_date = df.timestamp.max()
#     except:
#         max_date = df.timestamp.max()
#     max_date = pd.to_datetime(max_date).tz_convert('US/Eastern')
#     print("Max time found in database", max_date)
#     max_date = int(int(max_date.asm8)/1e6) # match date length of apis parameters which is 13 digits long
#     max_date = int(max_date + 1e3) #skip 1 minute past last max date + 1e3
#only call once per fetching session
def get_max_time(ex_name):
    global max_date
    with pd.HDFStore("crypto_prices.h5", fletcher32 = True, complevel = 9) as store:
        df = store["prices/max_minute"]
    try:
        if isinstance(df, pd.DataFrame):
            max_date = df.loc[ex_name,:][0]
    except:
        return("Please check dates")
    max_date = pd.to_datetime(max_date).tz_convert('US/Eastern')
    print("Max time found in database", max_date)
    max_date = int(int(max_date.asm8)/1e6) # match date length of apis parameters which is 13 digits long
    max_date = int(max_date + 1e4*6) #skip 1 minute past last max date + 1e4*6 (1e3 is one second)
    
def csv_logger(error):
    try:
        with open('error_logger_v2.1.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(error)
    except:
        pass
def date_to_unix(date):
    try:
        return int(int(pd.to_datetime(str(date)).tz_localize('US/Eastern').asm8)/1e6)
    except:
        return int(int(pd.to_datetime(str(date)).tz_convert('US/Eastern').asm8)/1e6)
    
def unix_to_date(unix):
    return pd.to_datetime(unix, unit = "ms").tz_localize('UTC').tz_convert('US/Eastern')

def backdays_to_unix(back_days):
    back_date = datetime.now() + timedelta(days = -back_days)
    back_date = str(int((back_date - datetime(1970,1,1)).total_seconds() *1e3)) #1e3 for ms
    return back_date

In [13]:
def get_crypto_prices(data, exchange, limit, ex_name):
    global not_found_ticker
    # global all_prices
    all_prices = pd.DataFrame()
    not_found_ticker = []
    csv_logger(str(pd.to_datetime(max_date, unit = "ms").tz_localize('UTC').tz_convert('US/Eastern')))
    for i in tqdm(data):
        try:
            temp_df = pd.DataFrame(exchange.fetch_ohlcv("%s/USD"%i, since = max_date, limit = limit, timeframe = "1m"), columns = ["timestamp", "Open","High", "Low", "Close", "Volume"])
            temp_df["ticker"] = "%s/USD"%i
        except Exception as err:
            csv_logger(err.args)
            try:
                temp_df = pd.DataFrame(exchange.fetch_ohlcv("%s/USDC"%i, since = max_date, limit = limit, timeframe = "1m"), columns = ["timestamp", "Open","High", "Low", "Close", "Volume"])
                temp_df["ticker"] = "%s/USDC"%i
            except Exception as err:
                csv_logger(err.args)
                try:
                    temp_df = pd.DataFrame(exchange.fetch_ohlcv("%s/USDT"%i, since = max_date, limit = limit, timeframe = "1m"), columns = ["timestamp", "Open","High", "Low", "Close", "Volume"])
                    temp_df["ticker"] = "%s/USDT"%i
                except Exception as err:
                    csv_logger(err.args)
                    temp_df = None
                    pass

            if (isinstance(temp_df,pd.DataFrame) == True):
                if len(temp_df) > 0:
                    all_prices = all_prices.append(temp_df,ignore_index=True)
                else: not_found_ticker.append(i); continue;
            else: not_found_ticker.append(i); continue;

    ##############
    #Final Result#
    ##############
    if len(all_prices) > 0 & (isinstance(all_prices,pd.DataFrame) == True):
        try:
            all_prices["timestamp"] = pd.to_datetime(all_prices["timestamp"], unit = "ms").apply(lambda x:x.tz_localize('UTC').tz_convert('US/Eastern'))
            all_prices["exchange"] = ex_name
            all_prices.reset_index(drop = True, inplace = True)
            if os.path.exists("crypto_prices.h5") == True:
                all_prices = all_prices.set_index("timestamp")
                with pd.HDFStore("crypto_prices.h5", fletcher32 = True, complevel = 9) as store:
                    max_date_df = store["prices/max_minute"]
                    max_date_df.loc[ex_name,:] = str(all_prices.index.max())
                    store.put("prices/max_minute", max_date_df, index = False, 
                              encoding = 'UTF-8',format = 'table') #append is FALSE/ replace the table
                with pd.HDFStore("crypto_prices.h5", fletcher32 = True, complevel = 9) as store:
                    store.put("prices/minute", all_prices, index = False, 
                              encoding = 'UTF-8',format = 'table',
                              append = True, min_itemsize={'exchange':9}) #CHECK DATA_COLS = T AND MIN_ITEM
            else:
                if os.path.exists("output.csv") == False: all_prices.head(0).to_csv("output.csv", mode='w', header=True)
                all_prices.to_csv("output.csv", mode='a', header=False)
                
                
            print("Tickers Found ", len(all_prices.ticker.unique()))
            print("Ticker not Found ", len(not_found_ticker))
        except Exception as err:
            csv_logger(err.args)
            print("The Exception raised is: " ,err)
    else:
        print("No data found!")
    print("Done on :",datetime.now())
    #pd.to_datetime(all_prices["timestamp"], unit = "ms").apply(lambda x:x.tz_localize('UTC').tz_convert('US/Eastern'))


In [14]:
########################
#if running first time:#
########################
# max_date = date_to_unix("2021-01-01 00:00:00-05:00")
#if just updating db use always:
get_max_time(ex4) 

while max_date < date_to_unix(datetime.now(timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M:%S")):
    # get_max_time(ex4)
    print(ex4)
    get_crypto_prices(data, kucoin, limit = 1500, ex_name=ex4)# all tickers first, pass data
    
    get_max_time(ex3)
    print(ex3)
    get_crypto_prices(not_found_ticker, mexc, limit = 1000, ex_name=ex3)

    get_max_time(ex1)
    print(ex1)
    get_crypto_prices(not_found_ticker, binanceus, limit = 1500, ex_name=ex1)

    # get_max_time(ex2)
    # print(ex2)
    # get_crypto_prices(not_found_ticker, kraken, limit = 1500, ex_name=ex2)
    
    
    get_max_time(ex4) #kucoin
    # break
    


Max time found in database 2021-12-28 22:16:00-05:00
kucoin


100%|██████████████████████████████████████████████████████████████████████████████| 8175/8175 [03:48<00:00, 35.79it/s]


Tickers Found  441
Ticker not Found  7664
Done on : 2022-10-16 12:40:32.260903
Max time found in database 2021-12-28 22:20:00-05:00
mexc


100%|██████████████████████████████████████████████████████████████████████████████| 7664/7664 [03:20<00:00, 38.16it/s]


No data found!
Done on : 2022-10-16 12:43:53.190998
Max time found in database 2021-11-02 08:00:00-04:00
binanceus


...

The Exception raised is:  HDF5 error back trace

  File "C:\ci\hdf5_1611496732392\work\src\H5Dio.c", line 314, in H5Dwrite
    can't write data
  File "C:\ci\hdf5_1611496732392\work\src\H5VLcallback.c", line 2186, in H5VL_dataset_write
    dataset write failed
  File "C:\ci\hdf5_1611496732392\work\src\H5VLcallback.c", line 2152, in H5VL__dataset_write
    dataset write failed
  File "C:\ci\hdf5_1611496732392\work\src\H5VLnative_dataset.c", line 207, in H5VL__native_dataset_write
    can't write data
  File "C:\ci\hdf5_1611496732392\work\src\H5Dio.c", line 781, in H5D__write
    can't write data
  File "C:\ci\hdf5_1611496732392\work\src\H5Dchunk.c", line 2728, in H5D__chunk_write
    unable to read raw data chunk
  File "C:\ci\hdf5_1611496732392\work\src\H5Dchunk.c", line 4034, in H5D__chunk_lock
    unable to preempt chunk(s) from cache
  File "C:\ci\hdf5_1611496732392\work\src\H5Dchunk.c", line 3700, in H5D__chunk_cache_prune
    unable to preempt one or more raw data cache entry
  Fi

100%|██████████████████████████████████████████████████████████████████████████████| 7660/7660 [03:47<00:00, 33.73it/s]


No data found!
Done on : 2022-10-16 21:16:51.738189
Max time found in database 2021-11-02 08:00:00-04:00
binanceus


 81%|███████████████████████████████████████████████████████████████▍              | 6226/7660 [21:20<04:55,  4.86it/s]


KeyboardInterrupt: 

#### Workspace for Checking 

In [13]:
# kucoin.fetch_ohlcvc("LTC/USDT",limit =100000, since = 1638334800000)
# pd.to_datetime(1639273800000, unit = "ms").tz_localize('UTC').tz_convert('US/Eastern')
# tickers = list(mexc.currencies.keys())pd.to_datetime(1639273800000, unit = "ms").tz_localize('UTC').tz_convert('US/Eastern')

In [151]:
# max_date = pd.read_csv("Firas_prices.csv", index_col=0).timestamp.max()
# max_date = pd.to_datetime(max_date).tz_convert('US/Eastern')
# print(max_date)
# max_date = int(int(max_date.asm8)/1e6)
# max_date = int(max_date + 1e3)

In [45]:
date_to_unix("2021-07-01 00:00:00-05:00")

1625115600000

In [54]:
import numpy as np
# df = pd.DataFrame(binanceus.fetch_ohlcv("BTC/USDT",since = int(max_date-1e3+6*1e4), limit= 1000, timeframe = "1m"), columns = ["timestamp", "Open","High", "Low", "Close", "Volume"])
# df["timestamp"] = pd.to_datetime(df["timestamp"], unit = "ms").apply(lambda x:x.tz_localize('UTC').tz_convert('US/Eastern'))
# df["ticker"] = "BTC/USDT"
df["exchange"] = np.nan
# x = df["timestamp"].max()

# x = pd.to_datetime(x.value, unit="ns")
# str(int((end - datetime.datetime(1970,1,1)).total_seconds()))

print(df["timestamp"].min(), df["timestamp"].max(), len(df))

2021-12-26 00:36:00-05:00 2021-12-26 17:15:00-05:00 1000


In [35]:
int(int(pd.to_datetime(str(df["timestamp"].max())).tz_convert('US/Eastern').asm8)/1e6)

1609567140000

In [105]:
# df.head(0).to_csv("Firas_prices_v2.csv", mode='w', header=True)